In [6]:
from typing import List
from spacy.tokens import Doc, Span
from utils import load_models, print_clusters, print_comparison

In [1]:
from torch.distributed import ProcessGroup

In [9]:
predictor, nlp = load_models()

EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
f = open("the_greek_interpreter.txt","r") 
the_greek_interpreter = f.read()
the_greek_interpreter_clusters = predictor.predict(the_greek_interpreter)['clusters']
doc = nlp(the_greek_interpreter)
f.close()

## The problem
AllenNLP coreference resolution models seems to find better clusters - numerous clusters that are usually more accurate than the ones found by Huggingface NeuralCoref model. However, the biggest problem lies in the next step - the step of replacing found mentions with the most meaningfull spans from each clusters (that we call the "heads"). We've found a couple of easy-to-fix problems which seem to lead to errors most often. Our ideas can be summed up as:
- not resolving coreferences in the clusters that doesn't contain any noun phrases (usually it comes down to the clusters composed only of pronouns),
- chosing the head of the cluster which is a noun phrase (isn't a pronoun),
- resolving only the inner span in the case of nested coreferent mentions.

We show all of our improvements by example - for more details please refer to our [third and last blog post](). If you're interested in problems themselves our [second blog post](https://neurosys.com/article/most-popular-frameworks-for-coreference-resolution/) contains all the definitions and theory.

## Original AllenNLP impelemntation of the `replace_corefs` method

We extract the main "logic" into the separate function that will be used in our every method as we leave the core of AllenNLP's logic untouched. So as for now, we will compare our solutions to the `original_replace_corefs` method implemented in AllenNLP `coref.py` (we've just copied it here explicitly in order to compare with the improved method we propose).

In [ ]:
def core_logic_part(document: Doc, coref: List[int], resolved: List[str], mention_span: Span):
    final_token = document[coref[1]]
    if final_token.tag_ in ["PRP$", "POS"]:
        resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
    else:
        resolved[coref[0]] = mention_span.text + final_token.whitespace_
    for i in range(coref[0] + 1, coref[1] + 1):
        resolved[i] = ""
    return resolved


def original_replace_corefs(document: Doc, clusters: List[List[List[int]]]) -> str:
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        mention_start, mention_end = cluster[0][0], cluster[0][1] + 1
        mention_span = document[mention_start:mention_end]

        for coref in cluster[1:]:
            core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

## Improvements
### Redundant clusters - lack of a meaningfull mention that could become the head
We completely ignore (we don't resove them at all) the clusters that doesn't contain any noun phrase.

In [ ]:
def get_span_noun_indices(doc: Doc, cluster: List[List[int]]) -> List[int]:
    spans = [doc[span[0]:span[1]+1] for span in cluster]
    spans_pos = [[token.pos_ for token in span] for span in spans]
    span_noun_indices = [i for i, span_pos in enumerate(spans_pos)
        if any(pos in span_pos for pos in ['NOUN', 'PROPN'])]
    return span_noun_indices

def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:  # if there is at least one noun phrase...
            mention_start, mention_end = cluster[0][0], cluster[0][1] + 1
            mention_span = document[mention_start:mention_end]

            for coref in cluster[1:]:
                core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

In [ ]:
with open("neural_the_greek_interpreter_1.txt","w") as f_write:
    f_write.write(improved_replace_corefs(doc, clusters))

**Example**  
We want to take our code and create a game. Let's remind ourselves how to do that.  

**Original coreference resolution clusters** (by AllenNLP)   
We --> We; our; 's; ourselves  
create --> create; that

In [ ]:
text = "We want to take our code and create a game. Let's remind ourselves how to do that."
clusters = predictor.predict(text)['clusters']
doc = nlp(text)

In [ ]:
print_comparison(original_replace_corefs(doc, clusters), improved_replace_corefs(doc, clusters))

### Cataphora problem - choosing the wrong cluster *head*
We redefine the span that becomes a cluster head. Instead of choosing the first mention in the cluster, we pick the one that is the first **noun phrase in the cluster** - we define it as the first span that contains a noun.

In [ ]:
def get_cluster_head(doc: Doc, cluster: List[List[int]], noun_indices: List[int]):
    head_idx = noun_indices[0]
    head_start, head_end = cluster[head_idx]
    head_span = doc[head_start:head_end+1]
    return head_span, [head_start, head_end]


def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)

            for coref in cluster:
                if coref != mention:  # we don't replace the head itself
                    core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

In [ ]:
with open("neural_the_greek_interpreter_2.txt","w") as f_write:
    f_write.write(improved_replace_corefs(doc, clusters))

**Example**   
"He is a great actor!", he said about John Travolta.

**Original coreference resolution clusters** (by AllenNLP)  
He --> He; John Travolta

In [ ]:
text = '"He is a great actor!", he said about John Travolta.'
clusters = predictor.predict(text)['clusters']
doc = nlp(text)

In [ ]:
print_comparison(original_replace_corefs(doc, clusters), improved_replace_corefs(doc, clusters))

### Nested coreferent mentions
I the case of nested mentions we choose to resolve the inner span (e.g. for the mention "his dog" the token *his* can be the inner span). That just means we don't want to resolve outer spans.

In [ ]:
def is_containing_other_spans(span: List[int], all_spans: List[List[int]]):
    return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])


def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)
    all_spans = [span for cluster in clusters for span in cluster]  # flattened list of all spans

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)

            for coref in cluster:
                if coref != mention and not is_containing_other_spans(coref, all_spans):
                    core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

In [ ]:
with open("neural_the_greek_interpreter_3.txt","w") as f_write:
    f_write.write(improved_replace_corefs(doc, clusters))

**Example**  
Anna likes Tom. Tom is Anna's brother. Her brother is tall.

**Original coreference resolution clusters** (by AllenNLP)  
Tom --> Tom; Tom; Her brother  
Anna --> Anna; Anna 's; Her

In [ ]:
text = "Anna likes Tom. Tom is Anna's brother. Her brother is tall."
clusters = predictor.predict(text)['clusters']
doc = nlp(text)

In [ ]:
print_comparison(original_replace_corefs(doc, clusters), improved_replace_corefs(doc, clusters))

Our last version of the `improved_replace_corefs` contains all of ours refinements. And that's it! You can now use it in your project with or without the intersection strategies (see the other Jupyter Notebook file). Good luck!